<!--
 * @file        6_camera.ipynb
 * @brief       This notebook provides camera data processing and analysis.
 *
 * @authors     Jaehwan Lee (idljh5529@gmail.com)
 *
 * @date        2025-08-13 Released by AI Lab, Hansung University
 *
-->

# 6. Camera

이번 실습에서는 자율주행 차량의 핵심 센서인 카메라 데이터 처리에 대해 학습합니다.

## 실습 목표
1. **OpenCV 기초**: 이미지 로딩, 표시, 기본 처리 방법 학습
2. **이미지 전처리**: 노이즈 제거, 대비 조정, 블러링 등 기본 이미지 처리
3. **고급 이미지 처리**: 이진화, 모폴로지, 엣지 검출, 윤곽선 검출
4. **차선 검출**: 실제 도로 이미지에서 차선 정보를 도출
5. **BEV 변환**: 카메라 캘리브레이션 정보를 활용한 Bird's Eye View 변환

## 카메라 센서의 중요성

### 1. 자율주행에서의 카메라 역할
- **차선 인식**: 차량이 주행해야 할 차선 검출
- **물체 검출**: 다른 차량, 보행자, 신호등 등 인식
- **거리 추정**: 스테레오 비전이나 단안 카메라 기반 거리 측정
- **상황 인식**: 교통 표지판, 신호등 상태 등 판단

### 2. 이미지 처리의 필요성
- **환경 적응**: 다양한 조명 조건(주간, 야간, 흐림 등)에서 안정적 동작
- **노이즈 제거**: 센서 노이즈 및 환경적 노이즈 제거
- **특징 추출**: 의미 있는 정보(차선, 물체 등) 추출
- **실시간 처리**: 빠른 의사결정을 위한 효율적 알고리즘

### 3. BEV 변환의 장점
- **직관적 표현**: 위에서 내려다본 시점으로 거리 및 방향 파악 용이
- **정확한 측정**: 실제 거리 기반의 정확한 측정
- **센서 융합**: 다른 센서(LiDAR, 레이더) 데이터와 통합 용이

## 실습 데이터
- **다양한 예시 이미지**: 이미지 전처리 실습을 위한 기본적인 여러 예시 이미지들
- **다양한 환경의 차량 카메라 이미지**: 주간, 야간에서 취득한 차량 카메라 이미지
- **도로 이미지**: 차선 검출을 위한 실제 도로 환경
- **카메라 캘리브레이션 정보**: 내부/외부 파라미터

## 사용할 라이브러리
- **OpenCV**: 컴퓨터 비전 및 이미지 처리
- **NumPy**: 수치 연산 및 배열 처리
- **Matplotlib**: 결과 시각화

In [ ]:
# 외부에 정의된 파이썬 모듈(.py 파일)을 수정할 때마다 매번 커널을 재시작하지 않아도 변경 사항이 자동으로 반영되도록 설정
%load_ext autoreload
%autoreload 2

In [ ]:
# 필요한 라이브러리 임포트
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import warnings
warnings.filterwarnings('ignore')

# 사용자 정의 모듈 임포트
sys.path.append('./tutlibs/camera')
from tutlibs.camera.custom_camera import CameraProcessor

print("라이브러리 임포트 완료!")
print("Python 버전:", sys.version)
print("OpenCV 버전:", cv2.__version__)
print("NumPy 버전:", np.__version__)
print("Matplotlib 버전:", plt.matplotlib.__version__)


## 1. OpenCV 기초

먼저 OpenCV의 기본적인 이미지 처리 방법을 학습합니다.
이미지를 로드하고, 다양한 형태로 표시하는 방법을 실습합니다.

### 학습 목표
- 이미지 파일 로딩 방법
- 컬러 공간 변환 (BGR ↔ RGB ↔ Gray)
- matplotlib을 이용한 이미지 표시


In [ ]:
# 카메라 처리 객체 생성
camera_processor = CameraProcessor()

# 단일 이미지 로딩 테스트
print("=== 단일 이미지 로딩 테스트 ===")

# BGR 형태로 이미지 로딩 (OpenCV 기본)
sample_image_bgr = camera_processor.load_image("./../data/camera/vehicle/vehicle_camera_4.png", color_mode='BGR')
print(f"BGR 이미지 형태: {sample_image_bgr.shape}")

# RGB 형태로 이미지 로딩 (matplotlib 호환)
sample_image_rgb = camera_processor.load_image("./../data/camera/vehicle/vehicle_camera_4.png", color_mode='RGB')
print(f"RGB 이미지 형태: {sample_image_rgb.shape}")

# 그레이스케일로 이미지 로딩
sample_image_gray = camera_processor.load_image("./../data/camera/vehicle/vehicle_camera_4.png", color_mode='GRAY') # TODO: load_image 함수 완성
print(f"그레이스케일 이미지 형태: {sample_image_gray.shape}")

# 이미지들을 함께 표시
images = [sample_image_bgr, sample_image_rgb, sample_image_gray]
titles = ['BGR Image', 'RGB Image', 'Grayscale Image']
camera_processor.display_multiple_images(images, titles, cols=3, figsize=(18, 5))


In [ ]:
# 다양한 환경의 이미지 로딩
print("=== 다양한 환경의 이미지 로딩 ===")

# 여러 이미지가 저장된 폴더에서 로딩
practice_images_folder = "./../data/camera/practice"
vehicle_images_folder = "./../data/camera/vehicle"
practice_images = camera_processor.load_multiple_images(practice_images_folder)
vehicle_images = camera_processor.load_multiple_images(vehicle_images_folder)

print(f"로딩된 실습 이미지 수: {len(practice_images)}")
print(f"로딩된 차량 이미지 수: {len(vehicle_images)}")

### 실습 이미지 표시
for i, img in enumerate(practice_images):
    print(f"이미지 {i+1}: 크기 {img.shape}, 데이터 타입 {img.dtype}")

if len(practice_images) >= 6:
    display_images = practice_images[:6]
    titles = ['camera man', 'circuit', 'coins', 'colored chips', 'dandelion seed', 'coins']
else:
    display_images = practice_images
    titles = [f'Image {i+1}' for i in range(len(practice_images))]

camera_processor.display_multiple_images(display_images, titles, cols=3, figsize=(15, 10))

### 차량 이미지 표시
# 각 이미지의 기본 정보 출력
for i, img in enumerate(vehicle_images):
    print(f"이미지 {i+1}: 크기 {img.shape}, 데이터 타입 {img.dtype}")

# 다양한 환경의 이미지들을 표시
if len(vehicle_images) >= 4:
    display_images = vehicle_images[:4]
    titles = ['Day Time - Boulevard', 'Night Time - Tollgate', 'Night Time - Ramp', 'Day Time - Tollgate']
else:
    display_images = vehicle_images
    titles = [f'Image {i+1}' for i in range(len(vehicle_images))]

camera_processor.display_multiple_images(display_images, titles, cols=2, figsize=(15, 10))


## 2. 기본 이미지 처리

다양한 이미지들을 의미 있는 데이터로 변환하기 위해 여러 이미지 처리 기법들을 실습합니다.

### 처리할 항목들
1. **RGB to Grayscale**: 컬러 이미지를 그레이스케일로 변환
2. **노이즈 추가 및 제거**: 다양한 노이즈 모델링 및 필터링
3. **이미지 반전**: 밝기 반전 효과
4. **대비 조정**: 명암 대비 및 밝기 조정
5. **블러링 & 샤프닝**: 이미지 흐림 및 선명화
6. **모폴로지 연산**: 팽창(Dilation) 및 침식(Erosion)


In [ ]:
# 기본 이미지 처리 실습
print("=== 기본 이미지 처리 실습 ===")

# 실습용 이미지 선택 (마지막 차량 이미지 사용)
test_image = vehicle_images[-1].copy()

# 1. RGB to Grayscale 변환
print("1. RGB to Grayscale 변환")
gray_image = xxxxxx(test_image, xxxxxx) # TODO: cv2.cvtColor 사용 & https://docs.opencv.org/3.4/d8/d01/group__imgproc__color__conversions.html 참고

# 결과들을 함께 표시
images_to_show = [test_image, gray_image]


titles_to_show = [
    'Original', 'Grayscale'
]

camera_processor.display_multiple_images(images_to_show, titles_to_show, cols=2, figsize=(18, 12))

In [ ]:
# 실습용 이미지 선택
test_image = vehicle_images[1].copy()

# 2. 노이즈 추가
print("2. 노이즈 추가")
noisy_gaussian = camera_processor.add_noise(test_image, 'gaussian', 25) # TODO: add_noise 함수 완성
noisy_salt_pepper = camera_processor.add_noise(test_image, 'salt_pepper', 3) # TODO: add_noise 함수 완성
noisy_uniform = camera_processor.add_noise(test_image, 'uniform', 70) # TODO: add_noise 함수 완성

# 3. 노이즈 제거
print("3. 노이즈 제거")
denoised_gaussian = cv2.GaussianBlur(noisy_gaussian, (5, 5), 0)
denoised_salt_pepper = cv2.medianBlur(noisy_salt_pepper, 3)

# 결과들을 함께 표시
images_to_show = [test_image, noisy_gaussian, noisy_salt_pepper, noisy_uniform, denoised_gaussian, denoised_salt_pepper]


titles_to_show = [
    'Original', 'Gaussian Noise', 'Salt & Pepper Noise', 'Uniform Noise', 'Denoised Gaussian', 'Denoised Salt & Pepper'
]

camera_processor.display_multiple_images(images_to_show, titles_to_show, cols=3, figsize=(18, 8))

In [ ]:
# 실습용 이미지 선택
test_image = practice_images[3].copy()

# 4. 이미지 반전
print("4. 이미지 반전")
inverted_image = xxxxxx # TODO: bit 연산 중, 부정 연산 사용 (https://engineer-mole.tistory.com/237 참고)

# 결과들을 함께 표시
images_to_show = [test_image, inverted_image]


titles_to_show = [
    'Original', 'Inverted'
]

camera_processor.display_multiple_images(images_to_show, titles_to_show, cols=2, figsize=(12, 6))

In [ ]:
# 실습용 이미지 선택
test_image = practice_images[8].copy()

# 5. 대비 조정
print("5. 대비 조정")
high_contrast = cv2.convertScaleAbs(test_image, alpha=1.5, beta=30) # alpha (float): 대비 조정 (1.0은 원본, >1.0은 대비 증가), beta (int): 밝기 조정 (0은 원본, >0은 밝게)
low_contrast = cv2.convertScaleAbs(test_image, alpha=0.5, beta=-20) # alpha (float): 대비 조정 (1.0은 원본, <1.0은 대비 감소), beta (int): 밝기 조정 (0은 원본, <0은 어둡게)

# 결과들을 함께 표시
images_to_show = [test_image, high_contrast, low_contrast]


titles_to_show = [
    'Original', 'High Contrast', 'Low Contrast'
]

camera_processor.display_multiple_images(images_to_show, titles_to_show, cols=3, figsize=(12, 6))

In [ ]:
# 6. Contrast를 0부터 255로 정규화 - 히스토그램 스트레칭
print("6. Contrast에 대한 히스토그램 스트레칭")
stretched_image = cv2.normalize(xxxxxx, None, xxxxxx, xxxxxx, cv2.NORM_MINMAX) # TODO: 히스토그램 스트레칭 함수 인자 작성

# 결과들을 함께 표시
images_to_show = [test_image, stretched_image]

titles_to_show = ['Original', 'Stretched']

camera_processor.display_multiple_images(images_to_show, titles_to_show, cols=2, figsize=(12, 6))

# 원본 이미지의 히스토그램 확인
plt.figure(figsize=(18, 6))
plt.subplot(1, 2, 1)
plt.hist(test_image.ravel(), 256, [0, 256])
# min, max 값 확인 및 붉은 색으로 선 시각화
plt.axvline(x=test_image.min(), color='red', linestyle='--')
plt.axvline(x=test_image.max(), color='red', linestyle='--')
print(f"Original Image Min: {test_image.min()}, Max: {test_image.max()}")
plt.title('Original Histogram')

# 결과 이미지의 히스토그램 확인
plt.subplot(1, 2, 2)
plt.hist(stretched_image.ravel(), 256, [0, 256])
# min, max 값 확인 및 붉은 색으로 선 시각화
plt.axvline(x=stretched_image.min(), color='red', linestyle='--')
plt.axvline(x=stretched_image.max(), color='red', linestyle='--')
print(f"Stretched Image Min: {stretched_image.min()}, Max: {stretched_image.max()}")
plt.title('Stretched Histogram')
plt.show()

In [ ]:
# 실습용 이미지 선택
test_image = practice_images[4].copy()

# 7. 블러링/샤프닝 실습
print("7. 블러링/샤프닝 실습")

# 블러링 및 샤프닝
print("블러링 및 샤프닝")
blurred_image = camera_processor.blur_image(test_image, kernel_size=15) # TODO: blur_image 함수 완성
sharpened_image = camera_processor.sharpen_image(test_image)

# 결과 표시
noise_removal_images = [test_image, blurred_image, sharpened_image]


noise_removal_titles = [
    'Original', 'Blurred', 'Sharpened'
]

camera_processor.display_multiple_images(noise_removal_images, noise_removal_titles, cols=3, figsize=(15, 12))


In [ ]:
# 실습용 이미지 선택
test_image = practice_images[-1].copy()
# test_image = practice_images[5].copy() # TODO: 다양한 이미지에 적용
# test_image = practice_images[9].copy() # TODO: 다양한 이미지에 적용
# test_image = vehicle_images[-2].copy() # TODO: 다양한 이미지에 적용

# 모폴로지 연산 실습 (Dilation & Erosion)
print("=== 모폴로지 연산 실습 ===")

# 그레이스케일 이미지를 이진화하여 모폴로지 연산에 사용
binary_image = camera_processor.binarize_image(test_image, method='otsu')

# 팽창 및 침식 연산
dilated_image = camera_processor.dilate_image(binary_image, kernel_size=5, iterations=2) # 팽창: 이미지(1)를 팽창시키는 연산 - TODO
eroded_image = camera_processor.erode_image(binary_image, kernel_size=3, iterations=1) # 침식: 이미지(1)를 깎아내는 연산 - TODO

# 다양한 커널 크기로 실험
dilated_small = camera_processor.dilate_image(binary_image, kernel_size=2, iterations=1) # TODO
dilated_large = camera_processor.dilate_image(binary_image, kernel_size=7, iterations=3) # TODO
eroded_small = camera_processor.erode_image(binary_image, kernel_size=2, iterations=1) # TODO

morphology_images = [
    binary_image,
    dilated_small,
    dilated_image,
    dilated_large,
    eroded_small,
    eroded_image,
    xxxxxx(dilated_small, eroded_small)  # TODO:두 이미지의 차이 보기 - 논리합 사용 (https://engineer-mole.tistory.com/237 참고)
]

morphology_titles = [
    'Binary Original', 'Dilated (2x2, 1 iter)', 'Dilated (5x5, 2 iter)',
    'Dilated (7x7, 3 iter)', 'Eroded (2x2, 1 iter)', 'Eroded (3x3, 1 iter)',
    'Dilation - Erosion'
]

camera_processor.display_multiple_images(morphology_images, morphology_titles, cols=3, figsize=(18, 12))

# 모폴로지 연산의 효과 분석
print(f"\n=== 모폴로지 연산 효과 분석 ===")
print(f"원본 이진 이미지 - 흰색 픽셀 수: {np.sum(binary_image == 255)}")
print(f"팽창 후 - 흰색 픽셀 수: {np.sum(dilated_image == 255)} (증가량: {np.sum(dilated_image == 255) - np.sum(binary_image == 255)})")
print(f"침식 후 - 흰색 픽셀 수: {np.sum(eroded_image == 255)} (감소량: {np.sum(binary_image == 255) - np.sum(eroded_image == 255)})")


## 3. 고급 이미지 처리

이제 더 복잡한 이미지 처리 기법들을 학습합니다. 이 기법들은 차선 검출과 같은 고급 응용에 필수적입니다.

### 학습할 기법들
1. **이진화 (Binarization)**: 다양한 임계값 설정 방법
2. **모폴로지 연산**: Opening, Closing, Gradient 등
3. **엣지 검출**: Sobel, Canny 엣지 검출기
4. **윤곽선 검출**: 객체의 경계 찾기

이러한 기법들은 실제 도로 환경에서 차선, 차량, 표지판 등을 검출하는 데 핵심적인 역할을 합니다.

In [ ]:
# 실습용 이미지 선택
test_image = practice_images[2].copy() # TODO: 다양한 이미지에 적용
# test_image = practice_images[7].copy() # TODO: 다양한 이미지에 적용
# test_image = vehicle_images[-1].copy() # TODO: 다양한 이미지에 적용
gray_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

# 이진화 실습 - 다양한 방법 비교
print("=== 이진화 실습 ===")

# 다양한 이진화 방법 적용
binary_simple = camera_processor.binarize_image(gray_image, threshold=127, method='simple') # TODO: binarize_image 함수 완성
binary_adaptive_mean = camera_processor.binarize_image(gray_image, method='adaptive_mean') # TODO: binarize_image 함수 완성
binary_adaptive_gaussian = camera_processor.binarize_image(gray_image, method='adaptive_gaussian') # TODO: binarize_image 함수 완성
binary_otsu = camera_processor.binarize_image(gray_image, method='otsu') # TODO: binarize_image 함수 완성

# 결과 비교
binarization_images = [
    gray_image,
    binary_simple,
    binary_adaptive_mean,
    binary_adaptive_gaussian,
    binary_otsu,
    cv2.bitwise_and(gray_image, binary_otsu)  # 원본에 마스크 적용
]

binarization_titles = [
    'Original Grayscale', 'Simple (T=127)', 'Adaptive Mean',
    'Adaptive Gaussian', 'Otsu', 'Otsu Applied to Original'
]

camera_processor.display_multiple_images(binarization_images, binarization_titles, cols=3, figsize=(18, 12))

# 각 방법의 이진화 비율 계산
print(f"\n=== 이진화 결과 분석 ===")
total_pixels = gray_image.shape[0] * gray_image.shape[1]
print(f"Simple 이진화 - 흰색 비율: {np.sum(binary_simple == 255) / total_pixels * 100:.1f}%")
print(f"Adaptive Mean - 흰색 비율: {np.sum(binary_adaptive_mean == 255) / total_pixels * 100:.1f}%")
print(f"Adaptive Gaussian - 흰색 비율: {np.sum(binary_adaptive_gaussian == 255) / total_pixels * 100:.1f}%")
print(f"Otsu - 흰색 비율: {np.sum(binary_otsu == 255) / total_pixels * 100:.1f}%")


In [ ]:
# 고급 모폴로지 연산 실습
print("=== 고급 모폴로지 연산 실습 ===")

# 이진 이미지에 다양한 모폴로지 연산 적용
morph_opening = camera_processor.morphology_operations(binary_otsu, 'opening', 5) # TODO: morphology_operations 함수 완성
morph_closing = camera_processor.morphology_operations(binary_otsu, 'closing', 5) # TODO: morphology_operations 함수 완성
morph_gradient = camera_processor.morphology_operations(binary_otsu, 'gradient', 3) # TODO: morphology_operations 함수 완성
morph_tophat = camera_processor.morphology_operations(binary_otsu, 'tophat', 9) # TODO: morphology_operations 함수 완성
morph_blackhat = camera_processor.morphology_operations(binary_otsu, 'blackhat', 9) # TODO: morphology_operations 함수 완성

# 결과 비교
morphology_advanced_images = [
    binary_otsu,
    morph_opening,
    morph_closing,
    morph_gradient,
    morph_tophat,
    morph_blackhat
]

morphology_advanced_titles = [
    'Original Binary', 'Opening', 'Closing',
    'Gradient', 'Top Hat', 'Black Hat'
]

camera_processor.display_multiple_images(morphology_advanced_images, morphology_advanced_titles, cols=3, figsize=(18, 12))

# 각 연산의 특징 설명
print(f"\n=== 모폴로지 연산 특징 ===")
print("Opening: 노이즈 제거, 작은 객체 제거")
print("Closing: 구멍 메우기, 객체 연결")
print("Gradient: 엣지 검출, 객체 경계")
print("Top Hat: 밝은 영역 강조")
print("Black Hat: 어두운 영역 강조")


Sobel filter

<img src="../resources/ch6/sobel_filter.png" width="30%">

In [ ]:
# 엣지 검출 실습 (Sobel & Canny)
print("=== 엣지 검출 실습 ===")

# Sobel 엣지 검출
sobel_x, sobel_y, sobel_combined = camera_processor.edge_detection_sobel(test_image) # TODO: edge_detection_sobel 함수 완성

# Canny 엣지 검출 (다양한 임계값으로)
canny_50_150 = camera_processor.edge_detection_canny(test_image, 50, 150) # TODO: edge_detection_canny 함수 완성
canny_100_200 = camera_processor.edge_detection_canny(test_image, 100, 200) # TODO: edge_detection_canny 함수 완성
canny_30_100 = camera_processor.edge_detection_canny(test_image, 30, 100) # TODO: edge_detection_canny 함수 완성

# 결과 비교
edge_images = [
    gray_image,
    sobel_x,
    sobel_y,
    sobel_combined,
    canny_30_100,
    canny_50_150,
    canny_100_200,
    cv2.bitwise_or(sobel_combined, canny_50_150)  # Sobel + Canny 결합
]

edge_titles = [
    'Original Gray', 'Sobel X', 'Sobel Y', 'Sobel Combined',
    'Canny (30,100)', 'Canny (50,150)', 'Canny (100,200)', 'Sobel + Canny'
]

camera_processor.display_multiple_images(edge_images, edge_titles, cols=4, figsize=(20, 10))

# 엣지 검출 결과 분석
print(f"\n=== 엣지 검출 결과 분석 ===")
print(f"Sobel Combined - 엣지 픽셀 수: {np.sum(sobel_combined > 0)}")
print(f"Canny (50,150) - 엣지 픽셀 수: {np.sum(canny_50_150 > 0)}")
print(f"Canny (100,200) - 엣지 픽셀 수: {np.sum(canny_100_200 > 0)}")
print(f"Canny (30,100) - 엣지 픽셀 수: {np.sum(canny_30_100 > 0)}")

print("\n임계값이 낮을수록 더 많은 엣지가 검출되지만 노이즈도 증가합니다.")


In [ ]:
# 윤곽선 검출 실습
print("=== 윤곽선 검출 실습 ===")

# 이진 이미지에서 윤곽선 검출
contours, contour_image = camera_processor.detect_contours(binary_otsu) # TODO: detect_contours 함수 완성

# 정리된 이진 이미지에서 윤곽선 검출
clean_binary = camera_processor.morphology_operations(binary_otsu, 'opening', 3) # TODO: morphology_operations 함수 완성
contours_clean, contour_image_clean = camera_processor.detect_contours(clean_binary) # TODO: detect_contours 함수 완성

print(f"원본 이진 이미지에서 검출된 윤곽선 수: {len(contours)}")
print(f"정리된 이진 이미지에서 검출된 윤곽선 수: {len(contours_clean)}")

# 큰 윤곽선만 필터링
area_threshold = 100
large_contours = [cnt for cnt in contours_clean if cv2.contourArea(cnt) > xxxxxx] # TODO: 윤곽선 면적 필터링
print(f"큰 윤곽선 (면적 > 100) 개수: {len(large_contours)}")

# 큰 윤곽선만 그리기
large_contour_image = cv2.cvtColor(clean_binary, xxxxxx) # TODO: 그레이스케일 이미지를 3채널로 변환
cv2.drawContours(large_contour_image, large_contours, -1, (0, 255, 0), 2)

# 윤곽선 분석 (면적, 둘레, 경계 사각형)
contour_analysis_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB).copy()
# for i, contour in enumerate(large_contours[:10]):  # 상위 10개만
for i, contour in enumerate(contours[:10]):  # 상위 10개만
    # 면적과 둘레 계산
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    # 경계 사각형
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(contour_analysis_image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # 정보 텍스트
    cv2.putText(contour_analysis_image, f'A:{int(area)}', (x, y-10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

# 결과 표시
contour_images = [
    binary_otsu,
    cv2.cvtColor(contour_image, cv2.COLOR_BGR2RGB),
    clean_binary,
    cv2.cvtColor(contour_image_clean, cv2.COLOR_BGR2RGB),
    cv2.cvtColor(large_contour_image, cv2.COLOR_BGR2RGB),
    contour_analysis_image
]

contour_titles = [
    'Original Binary', 'All Contours', 'Cleaned Binary',
    'Cleaned Contours', 'Large Contours Only', 'Contour Analysis'
]

camera_processor.display_multiple_images(contour_images, contour_titles, cols=3, figsize=(18, 12))

# 윤곽선 통계
if large_contours:
    areas = [cv2.contourArea(cnt) for cnt in large_contours]
    print(f"\n=== 윤곽선 통계 ===")
    print(f"가장 큰 윤곽선 면적: {max(areas):.1f}")
    print(f"평균 윤곽선 면적: {np.mean(areas):.1f}")
    print(f"윤곽선 면적 범위: {min(areas):.1f} ~ {max(areas):.1f}")


## 4. 차선 검출 (Lane Detection)

이제 앞서 학습한 이미지 처리 기법들을 활용하여 실제 도로 이미지에서 차선을 검출하는 파이프라인을 구현합니다.

### 차선 검출 파이프라인
1. **전처리**: 그레이스케일 변환, 노이즈 제거
2. **엣지 검출**: Canny 엣지 검출기 적용
3. **관심 영역 설정**: 도로 영역만 마스킹
4. **직선 검출**: 허프 변환(Hough Transform)으로 직선 찾기
5. **후처리**: 검출된 직선을 원본 이미지에 오버레이

### 학습자 실습 목표
- 각 단계별로 파라미터를 조정해가며 최적의 결과 도출
- 다양한 조명 조건에서의 차선 검출 성능 비교
- 차선 검출 알고리즘의 한계점 및 개선 방안 분석


In [ ]:
# 기본 차선 검출 실습
print("=== 기본 차선 검출 실습 ===")

# 각 이미지에 대해 차선 검출 적용
for i, image in enumerate(vehicle_images[-8:]):
    print(f"\n--- 이미지 {i+1} 차선 검출 ---")
    
    # 기본 차선 검출 수행
    processed_edges, lane_result, results = camera_processor.lane_detection_pipeline(image) # TODO: lane_detection_pipeline 함수 완성
    
    # 결과 표시
    results = [image, processed_edges, lane_result]
    
    titles = [f'Original Image {i+1}', f'Processed Edges {i+1}', f'Lane Detection {i+1}']
    
    camera_processor.display_multiple_images(results, titles, cols=3, figsize=(18, 6))
    
    # 간단한 성능 분석
    edge_pixels = np.sum(processed_edges > 0)
    total_pixels = processed_edges.shape[0] * processed_edges.shape[1]
    edge_ratio = edge_pixels / total_pixels * 100
    
    print(f"엣지 픽셀 비율: {edge_ratio:.2f}%")
    print(f"관심 영역 크기: {processed_edges.shape}")


In [ ]:
# 단계별 차선 검출 파이프라인 분석
print("=== 단계별 차선 검출 파이프라인 분석 ===")

# 첫 번째 이미지로 상세 분석
analysis_image = vehicle_images[2].copy()

print("차선 검출 파이프라인의 각 단계를 분석합니다...")

# 단계별 처리 결과 획득
processed_edges, lane_result, pipeline_results = camera_processor.lane_detection_pipeline(analysis_image) # TODO: lane_detection_pipeline 함수 완성

# 각 단계 결과를 순서대로 표시
step_images = []
step_titles = []

for step_name, step_image in pipeline_results.items():
    if step_name == 'original':
        step_images.append(cv2.cvtColor(step_image, cv2.COLOR_BGR2RGB))
    elif step_name in ['gray', 'blurred', 'edges', 'masked']:
        step_images.append(step_image)
    elif step_name in ['lines', 'final']:
        step_images.append(cv2.cvtColor(step_image, cv2.COLOR_BGR2RGB))
    
    # 제목을 아래와 같이 변경
    title_mapping = {
        'original': '1. original image',
        'gray': '2. gray image',
        'blurred': '3. blurred image',
        'edges': '4. edges image',
        'masked': '5. masked image',
        'lines': '6. lines image',
        'final': '7. final image'
    }
    step_titles.append(title_mapping.get(step_name, step_name))

# 모든 단계를 하나의 그리드로 표시
camera_processor.display_multiple_images(step_images, step_titles, cols=4, figsize=(20, 12))

# 각 단계별 통계 정보
print(f"\n=== 각 단계별 분석 ===")
print(f"원본 이미지 크기: {pipeline_results['original'].shape}")
print(f"그레이스케일 이미지 평균 밝기: {np.mean(pipeline_results['gray']):.1f}")
print(f"Canny 엣지 검출된 픽셀 수: {np.sum(pipeline_results['edges'] > 0)}")
print(f"마스킹 후 엣지 픽셀 수: {np.sum(pipeline_results['masked'] > 0)}")

# 마스킹 효과 분석
masking_efficiency = np.sum(pipeline_results['masked'] > 0) / np.sum(pipeline_results['edges'] > 0) * 100
print(f"마스킹 효율성: {masking_efficiency:.1f}% (관심 영역에 집중)")


## 5. BEV 변환 (Bird's Eye View / Inverse Perspective Mapping)

카메라 캘리브레이션 정보를 활용하여 원근감이 있는 카메라 이미지를 위에서 내려다본 시점(BEV)으로 변환합니다.

### BEV 변환의 수학적 배경

#### 1. 카메라 모델
카메라는 핀홀 모델로 근사할 수 있으며, 월드 좌표계의 점 **P_w**가 이미지 좌표계의 점 **p**로 투영되는 과정은 다음과 같습니다:

**p = K [R|t] P_w**

여기서:
- **K**: 내부 파라미터 행렬 (Intrinsic Matrix)
- **[R|t]**: 외부 파라미터 행렬 (Extrinsic Matrix)
- **R**: 회전 행렬 (3×3)
- **t**: 평행이동 벡터 (3×1)

#### 2. 내부 파라미터 행렬 K
```
K = [fx  0  cx]
    [0  fy  cy]
    [0   0   1]
```
- **fx, fy**: 초점 거리 (픽셀 단위)
- **cx, cy**: 주점 (Principal Point) 좌표

#### 3. BEV 변환 원리
지면 위의 점들 (Z = 0)에 대해:
1. **이미지 → 월드**: 지면 조건을 이용한 역투영
2. **월드 → BEV**: 직교 투영으로 위에서 내려다본 뷰 생성

#### 4. 지면 역투영 수식
이미지 점 **(u, v)**가 지면 위의 점 **(X, Y, 0)**에 대응될 때:

**λ = (Z_ground + R⁻¹[2,:]·t) / (R⁻¹[2,:]·K⁻¹·[u,v,1]ᵀ)**

**[X, Y, Z]ᵀ = R⁻¹(λ·K⁻¹·[u,v,1]ᵀ - t)**

### 실습 목표
- 카메라 캘리브레이션 파라미터 설정
- BEV 변환 행렬 생성
- 원본 이미지를 BEV로 변환
- BEV 이미지에서 차선 검출


In [ ]:
# 카메라 캘리브레이션 파라미터 설정
print("=== 카메라 캘리브레이션 파라미터 설정 ===")

# 내부 파라미터 행렬 (Intrinsic Matrix)
intrinsic_matrix = np.array([
    [1085.674464197818,   0.0, 970.8083862917935],  # fx, 0, cx
    [  0.0, 1085.346652238244, 571.7330860257647],  # 0, fy, cy  
    [  0.0,   0.0,   1.0]   # 0, 0, 1
], dtype=np.float32)

# 왜곡 계수 (Distortion Coefficients)
distortion_coeffs = np.array([-0.272997037779982, 0.080290395208785, 0.0, 0.0, 0.0], dtype=np.float32)

# 외부 파라미터 - 차량 뒤축 아래 바닥을 기준으로 설정
# 카메라가 차량 앞쪽에 장착되어 있음
camera_height = 1.8  # 카메라 높이 (미터)
camera_forward = 1.4  # 차량 뒤축에서 카메라까지의 전방 거리 (미터)

# 회전 행렬 - 차량 좌표계에서의 카메라 각도 설정
rotation_matrix = np.array([
    [ -0.0174524,  0.0000000,  0.9998477],
    [ -0.9998477, -0.0000000, -0.0174524],
    [ 0.0000000, -1.0000000,  0.0000000 ]
], dtype=np.float32)

tilt_angle_pitch = np.deg2rad(90)  # 90도를 라디안으로 변환
rotation_matrix_pitch = np.array([
    [np.cos(tilt_angle_pitch),  0.0, np.sin(tilt_angle_pitch)],
    [0.0,  1.0, 0.0],
    [-np.sin(tilt_angle_pitch), 0.0, np.cos(tilt_angle_pitch)]
], dtype=np.float32)

tilt_angle_yaw = -np.deg2rad(90)  # 90도를 라디안으로 변환
rotation_matrix_yaw = np.array([
    [np.cos(tilt_angle_yaw),  -np.sin(tilt_angle_yaw), 0.0],
    [np.sin(tilt_angle_yaw),  np.cos(tilt_angle_yaw), 0.0],
    [0.0,  0.0, 1.0]
], dtype=np.float32)

rotation_matrix = rotation_matrix @ rotation_matrix_pitch
rotation_matrix = rotation_matrix @ rotation_matrix_yaw

# 평행이동 벡터 - 차량 뒤축 기준 좌표계에서 카메라 위치
translation_vector = np.array([0.0, camera_forward, camera_height], dtype=np.float32)

print("내부 파라미터 행렬 K:")
print(intrinsic_matrix)
print(f"\n카메라 위치: 전방 {camera_forward}m, 높이 {camera_height}m")

# 카메라 처리기에 파라미터 설정
camera_processor.setup_camera_params(
    intrinsic_matrix, 
    distortion_coeffs, 
    rotation_matrix, 
    translation_vector
)

# 카메라 파라미터 검증
print("\n=== 카메라 파라미터 검증 ===")
print(f"초점 거리: fx={intrinsic_matrix[0,0]:.1f}, fy={intrinsic_matrix[1,1]:.1f}")
print(f"주점: cx={intrinsic_matrix[0,2]:.1f}, cy={intrinsic_matrix[1,2]:.1f}")
print(f"회전 행렬 행렬식: {np.linalg.det(rotation_matrix):.3f} (1에 가까워야 함)")
print(f"변환 벡터 크기: {np.linalg.norm(translation_vector):.3f}m")


In [ ]:
# BEV 변환 실습
print("=== BEV (Bird's Eye View) 변환 실습 ===")

# 마지막 자동차 실습 이미지로 BEV 변환 수행
bev_test_image = vehicle_images[-1].copy()

print("1. 원본 이미지 왜곡 보정")
# 왜곡 보정
undistorted_image = camera_processor.undistort_image(bev_test_image) # TODO: undistort_image 함수 완성

print("2. BEV 변환 행렬 생성")
# BEV 변환 파라미터 설정
bev_width = 15.0      # BEV 이미지의 실제 폭 (미터)
bev_height = 8.0    # BEV 이미지의 실제 높이 (미터)  
bev_resolution = 0.02  # BEV 해상도 (미터/픽셀)

# BEV 변환 행렬 생성
transform_matrix, bev_size = camera_processor.create_bev_transform(
    bev_test_image.shape[:2], bev_width, bev_height, bev_resolution # TODO: create_bev_transform 함수 완성
)

print("3. BEV 변환 적용")
# BEV 변환 적용
bev_image = camera_processor.apply_bev_transform(undistorted_image, xxxxxx, xxxxxx) # TODO: apply_bev_transform 함수 완성

# 결과 비교
bev_comparison_images = [bev_test_image, undistorted_image, bev_image]

bev_comparison_titles = ['Original', 'Undistorted', 'BEV']

camera_processor.display_multiple_images(bev_comparison_images, bev_comparison_titles, cols=3, figsize=(18, 8))

print(f"\n=== BEV 변환 결과 분석 ===")
print(f"원본 이미지 크기: {bev_test_image.shape[:2]}")
print(f"BEV 이미지 크기: {bev_image.shape[:2]}")
print(f"BEV 실제 영역: {bev_width}m × {bev_height}m")
print(f"BEV 해상도: {bev_resolution}m/pixel")
print(f"픽셀당 실제 크기: {bev_resolution*100:.0f}cm × {bev_resolution*100:.0f}cm")


In [ ]:
# BEV 이미지에서 차선 검출
print("=== BEV 이미지에서 차선 검출 ===")

print("BEV 이미지에서 차선을 검출합니다...")

# BEV 이미지에서 차선 검출
bev_lane_binary, bev_lane_result = camera_processor.detect_lanes_bev(bev_image) # TODO: detect_lanes_bev 함수 완성

# 결과 표시
bev_lane_images = [
    bev_image,
    bev_lane_binary,
    bev_lane_result
]

# Canny 엣지 검출
canny_edges = xxxxxx(xxxxxx, 50, 150) # TODO: Canny 함수 사용 - cv2.Canny

bev_lane_images.append(cv2.cvtColor(canny_edges, cv2.COLOR_BGR2RGB))

# 허프 변환으로 직선 검출
hough_threshold = xxxxxx
hough_min_line_length = xxxxxx
hough_max_line_gap = xxxxxx
lines = xxxxxx(canny_edges, 1, np.pi/180, threshold=hough_threshold, 
                        minLineLength=hough_min_line_length, maxLineGap=hough_max_line_gap) # TODO: 허프 변환 함수 사용 - cv2.HoughLinesP

# 검출된 직선을 원본 이미지에 그리기
line_image = np.zeros_like(bev_image)
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 1)

bev_lane_images.append(cv2.cvtColor(line_image, cv2.COLOR_BGR2RGB))

bev_lane_titles = [
    'BEV original', 'BEV binary', 'BEV lane detection', 'BEV lane detection (Hough)'
]

camera_processor.display_multiple_images(bev_lane_images, bev_lane_titles, cols=3, figsize=(18, 8))


# BEV 차선 검출 성능 분석
lane_pixels = np.sum(bev_lane_binary == 255)
total_bev_pixels = bev_lane_binary.shape[0] * bev_lane_binary.shape[1]
lane_coverage = lane_pixels / total_bev_pixels * 100

print(f"\n=== BEV 차선 검출 분석 ===")
print(f"검출된 차선 픽셀 수: {lane_pixels}")
print(f"차선 픽셀 비율: {lane_coverage:.2f}%")
print(f"실제 차선 면적: {lane_pixels * (bev_resolution**2):.2f} m²")

# 차선 폭 추정 (간단한 방법)
# 각 행에서 차선 픽셀의 분포를 분석
row_lane_counts = np.sum(bev_lane_binary == 255, axis=1)
avg_lane_width_pixels = np.mean(row_lane_counts[row_lane_counts > 0]) if np.any(row_lane_counts > 0) else 0
avg_lane_width_meters = avg_lane_width_pixels * bev_resolution

print(f"평균 검출 차선 폭: {avg_lane_width_pixels:.1f} 픽셀 ({avg_lane_width_meters:.2f} m)")